In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import requests
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from fredapi import Fred
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Dense, Dropout, Conv1D, MaxPooling1D, Bidirectional, Flatten, Input, LayerNormalization, MultiHeadAttention, Add, GlobalAveragePooling1D, Attention
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import ta
import warnings
warnings.filterwarnings("ignore")

# === API Keys ===
NEWS_API_KEY = "e7b02a6bae394d849381d33ee450abf4"
FRED_API_KEY = ""
fred = Fred(api_key=FRED_API_KEY)

# === Load Macroeconomic Data ===
def load_macro_data(start_date, end_date):
    indicators = {
        'FEDFUNDS': 'interest_rate',
        'CPIAUCSL': 'cpi',
        'INDPRO': 'industrial_production',
        'UNRATE': 'unemployment_rate'
    }

    macro_df = pd.DataFrame()
    for code, name in indicators.items():
        data = fred.get_series(code, start_date, end_date)
        macro_df[name] = data

    macro_df.index = pd.to_datetime(macro_df.index)
    macro_df = macro_df.resample('D').ffill()
    return macro_df

# === News Sentiment ===
def fetch_news_sentiment(stock_symbol, start_date, end_date):
    analyzer = SentimentIntensityAnalyzer()
    url = (
        f'https://newsapi.org/v2/everything?'
        f'q={stock_symbol}&'
        f'from={start_date}&'
        f'to={end_date}&'
        f'sortBy=publishedAt&'
        f'apiKey={NEWS_API_KEY}'
    )
    response = requests.get(url)
    articles = response.json().get('articles', [])

    news_list = []
    for article in articles:
        date = pd.to_datetime(article['publishedAt']).date()
        title = article['title'] or ""
        sentiment = analyzer.polarity_scores(title)['compound']
        news_list.append({'date': date, 'sentiment': sentiment})

    news_df = pd.DataFrame(news_list)
    if news_df.empty:
        return pd.Series(dtype='float64')

    daily_sentiment = news_df.groupby('date')['sentiment'].mean()
    daily_sentiment.index = pd.to_datetime(daily_sentiment.index)
    return daily_sentiment.resample('D').ffill()

# === Load Stock + Merge All ===
def load_data(stock_symbol, start_date, end_date):
    stock_df = yf.download(stock_symbol, start=start_date, end=end_date)
    stock_df.dropna(inplace=True)

    stock_df['rsi'] = ta.momentum.RSIIndicator(stock_df['Close']).rsi()
    stock_df['macd'] = ta.trend.MACD(stock_df['Close']).macd_diff()
    stock_df['sma'] = ta.trend.SMAIndicator(stock_df['Close']).sma_indicator()
    stock_df['ema'] = ta.trend.EMAIndicator(stock_df['Close']).ema_indicator()

    macro_df = load_macro_data(start_date, end_date)
    news_sentiment = fetch_news_sentiment(stock_symbol, start_date, end_date)
    stock_df['news_sentiment'] = news_sentiment

    data = stock_df[['Close', 'rsi', 'macd', 'sma', 'ema']].merge(
        macro_df, left_index=True, right_index=True, how='left')
    data['news_sentiment'] = stock_df['news_sentiment']
    data.fillna(method='ffill', inplace=True)
    data.dropna(inplace=True)

    scaler = MinMaxScaler()
    data_scaled = scaler.fit_transform(data)

    return data_scaled, scaler, data

# === Create Dataset ===
def create_dataset(data, time_step, forecast_horizon):
    X, y = [], []
    for i in range(len(data) - time_step - forecast_horizon):
        X.append(data[i:(i + time_step)])
        if forecast_horizon == 1:
            y.append(data[i + time_step, 0])
        else:
            y.append(np.mean(data[i + time_step:i + time_step + forecast_horizon, 0]))
    return np.array(X), np.array(y)

# === Model Architectures ===
def create_1day_model(input_shape):
    inputs = Input(shape=input_shape)
    x = Conv1D(64, kernel_size=3, activation='relu')(inputs)
    x = MaxPooling1D(pool_size=2)(x)
    x = Bidirectional(LSTM(64))(x)
    x = Dropout(0.3)(x)
    outputs = Dense(1)(x)
    return Model(inputs, outputs)

def create_1week_model(input_shape):
    inputs = Input(shape=input_shape)
    x = Bidirectional(LSTM(64, return_sequences=True))(inputs)
    x = Attention()([x, x])
    x = Flatten()(x)
    x = Dropout(0.3)(x)
    outputs = Dense(1)(x)
    return Model(inputs, outputs)

def create_1month_model(input_shape):
    inputs = Input(shape=input_shape)
    x = LSTM(64, return_sequences=True)(inputs)
    x = LSTM(64, return_sequences=True)(x)
    x = Attention()([x, x])
    x = Flatten()(x)
    x = Dropout(0.3)(x)
    outputs = Dense(1)(x)
    return Model(inputs, outputs)

def create_1year_model(input_shape, num_heads=2, ff_dim=64):
    inputs = Input(shape=input_shape)
    x = Conv1D(64, kernel_size=3, activation='relu', padding='same')(inputs)
    x = LayerNormalization(epsilon=1e-6)(x)
    attention_output = MultiHeadAttention(num_heads=num_heads, key_dim=64)(x, x)
    x = Add()([x, attention_output])
    x = LayerNormalization(epsilon=1e-6)(x)
    x = Dense(ff_dim, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(1)(x)
    x = GlobalAveragePooling1D()(x)
    return Model(inputs, x)

# === Evaluation ===
def evaluate_model(model, X_test, y_test, scaler, num_features, name):
    predicted = model.predict(X_test)
    y_test_reshaped = y_test.reshape(-1, 1)
    predicted_full = np.zeros((len(predicted), num_features))
    actual_full = np.zeros((len(y_test), num_features))
    predicted_full[:, 0] = predicted.flatten()
    actual_full[:, 0] = y_test_reshaped.flatten()

    predicted_prices = scaler.inverse_transform(predicted_full)[:, 0]
    actual_prices = scaler.inverse_transform(actual_full)[:, 0]

    print(f"{name} MSE: {mean_squared_error(actual_prices, predicted_prices):.4f}")
    print(f"{name} MAE: {mean_absolute_error(actual_prices, predicted_prices):.4f}")
    print(f"{name} R2: {r2_score(actual_prices, predicted_prices):.4f}")

    plt.figure(figsize=(10, 5))
    plt.plot(actual_prices, label='Actual')
    plt.plot(predicted_prices, label='Predicted')
    plt.title(f'{name} Prediction')
    plt.legend()
    plt.show()

# === Main ===
if __name__ == '__main__':
    stock = 'AAPL'
    start = '2010-01-01'
    end = '2024-12-31'
    time_step = 60

    data, scaler, processed_df = load_data(stock, start, end)
    num_features = data.shape[1]

    forecast_settings = {
        '1-Day': {'horizon': 1, 'model_fn': create_1day_model},
        '1-Week': {'horizon': 7, 'model_fn': create_1week_model},
        '1-Month': {'horizon': 30, 'model_fn': create_1month_model},
        '1-Year': {'horizon': 252, 'model_fn': create_1year_model}
    }

    for name, settings in forecast_settings.items():
        print(f"\n=== Training {name} Model ===")
        X, y = create_dataset(data, time_step, settings['horizon'])

        split = int(len(X) * 0.8)
        X_train, X_test = X[:split], X[split:]
        y_train, y_test = y[:split], y[split:]

        model = settings['model_fn'](X_train.shape[1:])
        model.compile(optimizer=Adam(0.001), loss='mean_squared_error')
        early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

        model.fit(X_train, y_train, epochs=20, batch_size=64,
                  validation_data=(X_test, y_test), callbacks=[early_stop], verbose=1)

        evaluate_model(model, X_test, y_test, scaler, num_features, name)
